In [ ]:
import re
import sys
import yt_dlp


In [ ]:
arg1 = sys.argv[1]
print(arg1)
video_url = arg1
output_dir = "video"
output_format = "m4a"
video_website = "https://www.youtube.com/watch?v="

In [ ]:
video_url = "4uuy_DEZToo"

In [ ]:
# 格式化时间
def time_format(duration):
    if duration is None:
        duration = 0
    duration = int(duration)
    hours, remaining_seconds = divmod(duration, 3600)
    minutes = remaining_seconds // 60
    remaining_seconds = remaining_seconds % 60
    if hours > 1:
        return '{:02}:{:02}:{:02}'.format(hours, minutes, remaining_seconds)
    else:
        return '{:02}:{:02}'.format(minutes, remaining_seconds)

In [ ]:
print(time_format(None))

In [ ]:
def write_log(log, suffix=None):
    if suffix:
        __builtins__.print(f"|{log}|{suffix}")
    else:
        __builtins__.print(f"|{log}")

In [ ]:
def convert_bytes(byte_size, units=None):
    if units is None:
        units = ['B', 'KB', 'MB', 'GB']
    if byte_size is None:
        byte_size = 0
    # 初始单位是字节
    unit_index = 0
    # 将字节大小除以1024，直到小于1024为止
    while byte_size > 1024 and unit_index < len(units) - 1:
        byte_size /= 1024.0
        unit_index += 1
    # 格式化结果并返回
    return f"{byte_size:.2f}{units[unit_index]}"

# 下载显示模块
def show_progress(stream):
    downloaded_bytes = convert_bytes(stream['downloaded_bytes']).rjust(9)
    total_bytes = convert_bytes(stream['total_bytes'])
    speed = convert_bytes(stream['speed'], ['b', 'kb', 'mb', 'gb']).rjust(9)
    if stream['status'] in ["downloading", "error"]:
        bar = stream['downloaded_bytes'] / stream['total_bytes'] * 100
        bar = f"{bar:.1f}" if bar == 100 else f"{bar:.2f}"
        bar = bar.rjust(5)
        eta = time_format(stream['eta']).ljust(8)
        print((f"\r{bar}%|{downloaded_bytes}\{total_bytes}|{speed}/s|{eta}"),end="")
    if stream['status'] == "finished":
        elapsed = time_format(stream['elapsed']).ljust(8)
        print((f"\r100.0%|{downloaded_bytes}\{total_bytes}|{speed}/s|{elapsed}"),end="")
        print("")

In [ ]:
print(convert_bytes(0))

In [ ]:
def download_video(video_url, output_dir, output_format, video_website, format_code=480, output_dir_name=""):
    def print(text):
        __builtins__.print(re.sub(r'\[.+?\]', '', text))
    print ("[asddf]ssd]")
    print("1234567890")
    if output_dir_name:
        video_write_log = f"{output_dir_name}|{video_url}"
    else:
        video_write_log = video_url
    if output_format == 'm4a':
        format_out = "bestaudio[ext=m4a]/best"   # 音频编码
    else:
        format_out = f'bestvideo[ext=mp4][height<={format_code}]+bestaudio[ext=m4a]/best'  # 视频编码
    ydl_opts = {
        'outtmpl': f'{output_dir}/{video_url}.{output_format}',  # 输出文件路径和名称
        'format': f'{format_out}',  # 指定下载的最佳音频和视频格式
        "quiet": True,
        "noprogress": True,
        "progress_hooks": [show_progress]
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([f'{video_website}{video_url}'])  # 下载指定视频链接的视频
        write_log(f"{video_write_log}下载成功")  # 写入下载成功的日志信息
    except Exception as e:
        write_log((f"{video_write_log}下载失败, 错误信息：{str(e)}").replace("ERROR: ", "").replace(f"{video_url}: ", ""))  # 写入下载失败的日志信息
        return video_url

In [ ]:
download_video(video_url, output_dir, output_format, video_website)

In [1]:
import os
import re

try:
    # Python3
    from http.server import SimpleHTTPRequestHandler

except ImportError:
    # Python 2
    from SimpleHTTPServer import SimpleHTTPRequestHandler


def copy_byte_range(infile, outfile, start=None, stop=None, bufsize=16*1024):
    """Like shutil.copyfileobj, but only copy a range of the streams.

    Both start and stop are inclusive.
    """
    if start is not None: infile.seek(start)
    while 1:
        to_read = min(bufsize, stop + 1 - infile.tell() if stop else bufsize)
        buf = infile.read(to_read)
        if not buf:
            break
        outfile.write(buf)


BYTE_RANGE_RE = re.compile(r'bytes=(\d+)-(\d+)?$')
def parse_byte_range(byte_range):
    """Returns the two numbers in 'bytes=123-456' or throws ValueError.

    The last number or both numbers may be None.
    """
    if byte_range.strip() == '':
        return None, None

    m = BYTE_RANGE_RE.match(byte_range)
    if not m:
        raise ValueError('Invalid byte range %s' % byte_range)

    first, last = [x and int(x) for x in m.groups()]
    if last and last < first:
        raise ValueError('Invalid byte range %s' % byte_range)
    return first, last


class RangeRequestHandler(SimpleHTTPRequestHandler):
    """Adds support for HTTP 'Range' requests to SimpleHTTPRequestHandler

    The approach is to:
    - Override send_head to look for 'Range' and respond appropriately.
    - Override copyfile to only transmit a range when requested.
    """
    def send_head(self):
        if 'Range' not in self.headers:
            self.range = None
            return SimpleHTTPRequestHandler.send_head(self)
        try:
            self.range = parse_byte_range(self.headers['Range'])
        except ValueError as e:
            self.send_error(400, 'Invalid byte range')
            return None
        first, last = self.range

        # Mirroring SimpleHTTPServer.py here
        path = self.translate_path(self.path)
        f = None
        ctype = self.guess_type(path)
        try:
            f = open(path, 'rb')
        except IOError:
            self.send_error(404, 'File not found')
            return None

        fs = os.fstat(f.fileno())
        file_len = fs[6]
        if first >= file_len:
            self.send_error(416, 'Requested Range Not Satisfiable')
            return None

        self.send_response(206)
        self.send_header('Content-type', ctype)

        if last is None or last >= file_len:
            last = file_len - 1
        response_length = last - first + 1

        self.send_header('Content-Range',
                         'bytes %s-%s/%s' % (first, last, file_len))
        self.send_header('Content-Length', str(response_length))
        self.send_header('Last-Modified', self.date_time_string(fs.st_mtime))
        self.end_headers()
        return f

    def end_headers(self):
        self.send_header('Accept-Ranges', 'bytes')
        return SimpleHTTPRequestHandler.end_headers(self)

    def copyfile(self, source, outputfile):
        if not self.range:
            return SimpleHTTPRequestHandler.copyfile(self, source, outputfile)

        # SimpleHTTPRequestHandler uses shutil.copyfileobj, which doesn't let
        # you stop the copying before the end of the file.
        start, stop = self.range  # set in send_head()
        copy_byte_range(source, outputfile, start, stop)


def serve_directory(directory='.', port=8000):
    """Serves the contents of the specified directory over HTTP,
    with support for 'Range:' headers to load portions of files.

    Args:
        directory (str): The path to the directory to serve (default: '.').
        port (int): The port number to serve on (default: 8000).
    """
    os.chdir(directory)
    handler = RangeRequestHandler
    httpd = SimpleHTTPRequestHandler(("", port), handler)
    print("Serving directory '%s' on port %d." % (directory, port))
    httpd.serve_forever()


In [4]:
serve_directory


<function __main__.serve_directory(directory='.', port=8000)>